# Asynchronous Python

- [RealPython - Async IO in Python: A Complete Walkthrough](https://realpython.com/async-io-python/)
- [Miguel Grinberg - Asynchronous Python for the Complete Beginner](https://speakerdeck.com/pycon2017/miguel-grinberg-asynchronous-python-for-the-complete-beginner)
    - [ [Video](https://www.youtube.com/watch?v=iG6fr81xHKA) ]
- [Yury Selivanov asyncawait and asyncio in Python 3 6 and beyond PyCon 2017](https://github.com/PyCon/2017-slides/blob/master/Yury%20Selivanov%20-%20async-await%20and%20asyncio%20in%20Python%203.6%20and%20beyond.pptx)
    - [ [Video](https://www.youtube.com/watch?v=2ZFFv-wZ8_g) ]
- [Morvan - 加速爬虫: 异步加载 Asyncio](https://morvanzhou.github.io/tutorials/data-manipulation/scraping/4-02-asyncio/)

## Asyncio

```python
import asyncio

async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
loop.close()
```

[<img src='img/asyncio-2.png' width=640 />](https://docs.python.org/3.6/library/asyncio-task.html#example-chain-coroutines)

### Steps

1. `async` both worker and main functions
2. `loop = get_event_loop()`
3. `loop.run_until_complete()`
4. `future_result = loop.create_task()`
5. `await asyncio.wait([future_result1, future_result2, ...])`

In [12]:
import asyncio


async def find_divisibles(inrange, div_by):
    """A time-comsuming job

    [description]
    """
    print(f'Finding nums in range {inrange} divisible by {div_by} ...')

    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
        if i % 50000 == 0:
            # the magic trick to make function really async:
            # hang over the processing time to the event loop
            await asyncio.sleep(0.000001)

    print(f'(Done with nums in range {inrange} divisible by {div_by})')
    return located


async def main():
    '''event loop'''

    div1 = loop.create_task(find_divisibles(5000000, 42713))
    div2 = loop.create_task(find_divisibles(1000000, 112))
    div3 = loop.create_task(find_divisibles(6000, 22))
    await asyncio.wait([div1, div2, div3])

    return div1, div2, div3

In [14]:
if __name__ == '__main__':
    try:
        # loop = asyncio.get_event_loop()
        # loop.set_debug(True)
        
        # https://stackoverflow.com/a/55409674/3744499
        # Jupyter is already running an event loop, 
        # so you don't need to start the event loop yourself in jupyter
        # and you can directly call await main(url)

        # d1, d2, d3 = loop.run_until_complete(main())
        d1, d2, d3 = await main()
        
        print(d1.result()[:5])
    except Exception as e:
        raise e
    finally:
        # You always want to close the loop.
        # loop.close()
        pass

Finding nums in range 50000000 divisible by 42713 ...
Finding nums in range 1000000 divisible by 112 ...
Finding nums in range 6000 divisible by 22 ...
(Done with nums in range 6000 divisible by 22)
(Done with nums in range 1000000 divisible by 112)
(Done with nums in range 50000000 divisible by 42713)
[0, 42713, 85426, 128139, 170852]


---

In [19]:
import asyncio
import time


async def job(t):                   # async 形式的功能
    print(f'Start job {t}')
    await asyncio.sleep(t**2)          # 等待 "t" 秒, 期间切换其他任务
    print(f'Job {t} takes {t**2} sec')


async def main(loop):                       # async 形式的功能
    tasks = [loop.create_task(job(t)) for t in range(1, 4)]  # 创建任务, 但是不执行
    await asyncio.wait(tasks)               # 执行并等待所有任务完成

t1 = time.time()
loop = asyncio.new_event_loop()             # 建立 loop
loop.run_until_complete(main(loop))         # 执行 loop
loop.close()                                # 关闭 loop

print("Async total time : ", time.time() - t1)

Start job 1
Start job 2
Start job 3
Job 1 takes 1 sec
Job 2 takes 4 sec
Job 3 takes 9 sec
Async total time :  9.007589101791382


### aiohttp